In [1]:
from coppertop.core import ensurePath, unload
# set the following to your paths
'/Users/david/repos/github/DangerMouseB/coppertop/src/std' >> ensurePath
'/Users/david/repos/github/DangerMouseB/coppertop/src/dm' >> ensurePath
from importlib import reload

In [2]:
# %%bones

# // import common functions - including drop, fields, inject, join, at:put:, log, filter, and, or, not, to, startsWith, etc
# // and common types - date, time, datetimetz, tz, num, index, offset, count, list, set, map, hash, deflist, matrix, vec, tensor, path, filename, pfn, str, str, 
# //     num, idx

# include std.common
# from std.common import *

# <:ytm: num(`ytm)>
# <:usYtms: {`date`_1m`_2m`_3m`_6m`_1y`_2y`_3y`_4y`_5y`_7y`_10y`_20y`_30y!(date,ytm,ytm,ytm,ytm,ytm,ytm,ytm,ytm,ytm,ytm,ytm,ytm,ytm)}>
# oldFields: `"Date"`"1 Mo"`"2 Mo"`"3 Mo"`"6 Mo"`"1 Yr"`"2 Yr"`"3 Yr"`"5 Yr"`"7 Yr"`"10 Yr"`"20 Yr"`"30 Yr"

# ytms: "./data" <:path> join ("us yields.csv" <:filename>) fromCsv(, <:usYtms>, )(,oldFields)

In [3]:
# imports

import datetime

from dm.core import _, join, startsWith, inject, to, take, tvarray, fields, fromCsv, drop, agg, at, atPut, filter, \
    stats, vec, check, different, sameShape


In [4]:
### load the ytm data

renames = {
    'Date'  : 'date',
    '1 Mo'  : '_1m',
    '2 Mo'  : '_2m',
    '3 Mo'  : '_3m',
    '6 Mo'  : '_6m',
    '1 Yr'  : '_1y',
    '2 Yr'  : '_2y',
    '3 Yr'  : '_3y',
    '5 Yr'  : '_5y',
    '7 Yr'  : '_7y',
    '10 Yr' : '_10y',
    '20 Yr' : '_20y',
    '30 Yr' : '_30y'
}


conversions = dict(
    date=vec.to(_, datetime.date, 'mm/dd/yy'),
    _1m=vec.to(_, float),
    _2m=vec.to(_, float),
    _3m=vec.to(_, float),
    _6m=vec.to(_, float),
    _1y=vec.to(_, float),
    _2y=vec.to(_, float),
    _3y=vec.to(_, float),
    _5y=vec.to(_, float),
    _7y=vec.to(_, float),
    _10y=vec.to(_, float),
    _20y=vec.to(_, float),
    _30y=vec.to(_, float),
)

path = './data'
filename = 'us yields.csv'
ytms = (path + '/' + filename) >> fromCsv(_, renames, conversions)


In [5]:
# %%bones

# include std.agg, std.wrangling
# from std.agg import agg, filter, fields, take, to     // imported as overloads to the ones in std.common
# from std.wrangling import diff
# from std.maths import cov

# ytms2: ytms fields drop `date`_2m inject(,agg(ytms),) {[p,f] 
#     p at: `log join (f drop 1) put: p f log
# }


In [6]:
# take logs

ytms2 = ytms \
    >> fields >> drop >> ['date', '_2m'] \
    >> inject(_, agg(ytms), _) >> (lambda prior, field:
        prior >> atPut(_,
            'log' >> join >> (field >> drop >> 1),
            prior >> at(_, field) >> vec.log
        )
    )

/Users/david/repos/github/DangerMouseB/coppertop/src/std/coppertop/std/vec.py:23: RuntimeWarning: divide by zero encountered in log
  return numpy.log(v)


In [7]:
# %%bones

# d1: 2021.01.01
# d2: 2021.04.01
# usD1ToD2: ytms2 filter {d1 <= x.date and x.date < d2}


In [8]:
# filter for the desired date range

d1 = '2021.01.01' >> to(_, datetime.date, 'yyyy.mm.dd')
d2 = '2021.04.01' >> to(_, datetime.date, 'yyyy.mm.dd')

usD1ToD2 = ytms2 >> filter >> (lambda r: d1 <= r.date and r.date < d2)

In [9]:
# %% bones

# usDiffs: usD1ToD2 fields drop(,`date) inject(,agg(),) {[p,f] p at: f put: usD1ToD2 f diff}
# usDiffCov: usDiffs take (usDiffs fields filter (startsWith(,`_))) to(,<:matrix>) cov
# usLogDiffCov: usDiffs take (usDiffs fields filter (startsWith(,`log))) to(,<:matrix>) cov


In [10]:
# diff and calc covariance matrices

usDiffs = usD1ToD2 \
    >> fields >> drop >> 'date' \
    >> inject(_, agg(), _) >> (lambda p, f:
        p >> atPut(_, f, usD1ToD2 >> at(_, f) >> vec.diff)
    )

usDiffCov = usDiffs \
    >> take >> (usDiffs >> fields >> filter >> (startsWith(_, '_'))) \
    >> to(_, tvarray) >> stats.core.cov

usLogDiffCov = usDiffs \
    >> take >> (usDiffs >> fields >> filter >> (startsWith(_, 'log'))) \
    >> to(_, tvarray) >> stats.core.cov



In [11]:
usDiffCov \
    >> check >> sameShape >> usLogDiffCov \
    >> check >> different >> usLogDiffCov


tvarray([[ 2.42424242e-04  5.96969697e-04  2.54545455e-04 ...  3.39393939e-04
  -8.18181818e-05  9.39393939e-05]
 [ 5.96969697e-04  1.88106061e-03  7.93181818e-04 ...  1.03257576e-03
  -3.50000000e-04  2.80303030e-04]
 [ 2.54545455e-04  7.93181818e-04  3.65909091e-04 ...  4.65909091e-04
  -7.72727273e-05  1.22727273e-04]
 ...
 [ 3.39393939e-04  1.03257576e-03  4.65909091e-04 ...  6.44696970e-04
  -1.68181818e-04  1.46969697e-04]
 [-8.18181818e-05 -3.50000000e-04 -7.72727273e-05 ... -1.68181818e-04
   4.27272727e-04 -8.18181818e-05]
 [ 9.39393939e-05  2.80303030e-04  1.22727273e-04 ...  1.46969697e-04
  -8.18181818e-05  8.78787879e-05]])